In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

num_samples = 10000
max_sequence_length = 20

input_texts = []
target_texts = []

for _ in range(num_samples):
    input_text = " ".join(np.random.choice(["hello", "goodbye", "how are you", "I love you"], size=3))
    target_text = " ".join(np.random.choice(["bonjour", "au revoir", "comment ça va", "je t'aime"], size=3))
    
    input_texts.append(input_text)
    target_texts.append(target_text)



2024-02-06 16:07:33.573491: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-06 16:07:33.598473: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 16:07:33.598493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 16:07:33.599119: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 16:07:33.602943: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-06 16:07:33.603491: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
tokenizer_input = Tokenizer()
tokenizer_input.fit_on_texts(input_texts)
input_sequences = tokenizer_input.texts_to_sequences(input_texts)

tokenizer_target = Tokenizer()
tokenizer_target.fit_on_texts(target_texts)
target_sequences = tokenizer_target.texts_to_sequences(target_texts)



In [3]:
encoder_input_data = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
decoder_input_data = pad_sequences(target_sequences, maxlen=max_sequence_length, padding='post')

decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, 0:-1] = decoder_input_data[:, 1:]
decoder_target_data[:, -1] = 0  


In [4]:
embedding_dim = 50
latent_dim = 256

encoder_inputs = Input(shape=(max_sequence_length,))
encoder_embedding = Embedding(input_dim=len(tokenizer_input.word_index) + 1, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True)(encoder_embedding)

decoder_inputs = Input(shape=(max_sequence_length,))
decoder_embedding = Embedding(input_dim=len(tokenizer_target.word_index) + 1, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True)(decoder_embedding)


In [5]:

attention = Attention()([encoder_lstm, decoder_lstm])
context = tf.keras.layers.Concatenate(axis=-1)([decoder_lstm, attention])

output = Dense(len(tokenizer_target.word_index) + 1, activation='softmax')(context)


In [6]:
model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=output)

In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10


W0000 00:00:1707217657.267667   33597 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 3000 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 12582912 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


125/125 [==============================] - ETA: 0s - loss: 0.7035 - accuracy: 0.7667

W0000 00:00:1707217661.837677   33597 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 3000 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 12582912 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


125/125 [==============================] - 7s 38ms/step - loss: 0.7035 - accuracy: 0.7667 - val_loss: 0.5517 - val_accuracy: 0.7876
Epoch 2/10
125/125 [==============================] - 4s 35ms/step - loss: 0.4126 - accuracy: 0.8619 - val_loss: 0.1910 - val_accuracy: 0.9515
Epoch 3/10
125/125 [==============================] - 4s 34ms/step - loss: 0.0718 - accuracy: 0.9814 - val_loss: 0.0219 - val_accuracy: 0.9973
Epoch 4/10
125/125 [==============================] - 5s 36ms/step - loss: 0.0104 - accuracy: 0.9997 - val_loss: 0.0046 - val_accuracy: 1.0000
Epoch 5/10
125/125 [==============================] - 4s 35ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 6/10
125/125 [==============================] - 4s 35ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 7/10
125/125 [==============================] - 4s 36ms/step - loss: 0.0119 - accuracy: 0.9972 - val_loss: 0.0102 - val_accuracy: 1.0000
Epoch 8/10